In [ ]:
#%pip install nvidia-modulus nvidia-modulus-sym==1.6.0

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

from modulus.sym.hydra import instantiate_arch, ModulusConfig
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra.utils import compose

from modulus.sym.solver import Solver
from modulus.sym.domain import Domain
from modulus.sym.geometry.primitives_2d import Rectangle
#from modulus.sym.geometry.primitives_1d import Line1D
from modulus.sym.domain.constraint import PointwiseBoundaryConstraint, PointwiseInteriorConstraint
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.node import Node
from modulus.sym.key import Key
from modulus.sym.eq.pde import PDE
from sympy import Symbol, Function, Eq

from modulus.sym.models.fully_connected import FullyConnectedArch
from modulus.sym.eq.pde import PDE

# https://docs.nvidia.com/deeplearning/modulus/modulus-sym/user_guide/notebook/notebook.html

In [ ]:
c = 2

# make ldc domain
x, y = Symbol("x"), Symbol("y")
rec = Rectangle((0,0), (2,1))
ldc_domain = Domain()

In [ ]:
samples = rec.sample_boundary(100, quasirandom=True)
plt.figure()
plt.scatter(samples['x'], samples['y'], label='Signed Distance Field')
plt.legend()
plt.show()

samples = rec.sample_interior(100, quasirandom=True)
plt.figure()
plt.scatter(samples['x'], samples['y'], label='Signed Distance Field')
plt.legend()
plt.show()

In [ ]:
# Définir le réseau
net = FullyConnectedArch(
        input_keys=[Key("x"),Key("y")], output_keys=[Key("u")], nr_layers=3, layer_size=32
    )

nodes_net = [net.make_node(name="network")]

In [ ]:
class CustomPDE(PDE):
    def __init__(self):
        # Définir l'équation de la chaleur
        u = Function('u')(x, y)
        self.equations = {}
        self.equations['heat_eq'] = u.diff(y) - c**2 * u.diff(x, 2)
        
        self.equations['init_eq'] = u - (x**2*(2-x))

eq = CustomPDE()
nodes_pde = eq.make_nodes()

In [ ]:
nodes = nodes_net + nodes_pde

In [ ]:
nbBounds = 500

# left wall
left_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 0.0},
    batch_size=nbBounds,
    criteria=Eq(x, 0),
)
ldc_domain.add_constraint(left_wall, "left_wall")

# right wall
right_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"u": 0.0},
    batch_size=nbBounds,
    criteria=Eq(x, 2),
)
ldc_domain.add_constraint(right_wall, "right_wall")

# initial condition
init_wall = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=rec,
    outvar={"init_eq": 0.0},
    batch_size=nbBounds,
    criteria=Eq(y, 0),
)
ldc_domain.add_constraint(init_wall, "init_wall")

In [ ]:
nbCollocation = 1500

# Ajouter les contraintes
ldc_domain.add_constraint(
    PointwiseInteriorConstraint(
        nodes=nodes, 
        geometry=rec, 
        outvar={'heat_eq': 0.0},
        batch_size=nbCollocation,
        ), 
    'interior'
    )

In [ ]:
xx, yy = np.meshgrid(np.linspace(0, 2,100), 
                     np.linspace(0, 1,100))
# add inferencer
inference = PointwiseInferencer(
    nodes=nodes,
    invar={"x": xx.reshape(-1,1),
           "y": yy.reshape(-1,1)
           },
    output_names=["u"],
)

ldc_domain.add_inferencer(inference, "inf_data")

In [ ]:
cfg = compose(config_path=".", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints

# Définir le solveur
solver = Solver(cfg=cfg, domain=ldc_domain)

# Entraîner le modèle
solver.solve()

In [ ]:
data = np.load('./outputs/inferencers/inf_data.npz', allow_pickle=True)
data = np.atleast_1d(data.f.arr_0)[0]

data['x'].shape,data['y'].shape,data['u'].shape
plt.imshow(data['u'].reshape((100,100)),cmap='rainbow')
plt.colorbar()